In [1]:
# Tensor product, q = 3 matrix Maxwell, Dirac and differential forms:

# First some helper code:
# define our root of unity:
var('w')
W = exp(I*2*pi/3)
k = CyclotomicField(3)
Z = k.gen()

# Choose rotation/Lorentz type for p.T terms:
use_rotation = False

# Define symbols to help with walking the parse tree:
var('sym1 sym2')

# define our w_modulus function:
def w_mod(r):
    r = expand(r)
    try:
        if len(r.operands()) == 0:
            return r
    except:
        return r
    if r.operator() == (sym1 + sym2).operator():
        result = 0
        for term in r.operands():
            result += w_mod(term)
        return result
    if r.operator() == (sym1*sym2).operator():
        result = 1
        for term in r.operands():
            result *= w_mod(term)
        return result
    if r.operator() == (sym1^sym2).operator():
        head = r.operands()[0]
        tail = r.operands()[1]
        if head != w:
            return r
        try:
            tail = Mod(tail, 3)
            return head^tail
        except:
            return r

# wrapper for our w_mod function, so we can apply it to matrices:
def matrix_w_mod(A):
    return A.apply_map(w_mod)


# Implement 1 + w + w^2 = 0
# Needs verifying!
def expand_w(r):
    r = expand(r)
    try:
        if r.operator() != (sym1 + sym2).operator():  # Already compact, so return it unchanged
            return r
    except:
        return r
    non_w2_terms = r.substitute({w^2: 0})
    w2_terms = r - non_w2_terms
    w2_terms = expand(w2_terms.substitute({w: 1}))
    r2 = expand(non_w2_terms - w2_terms*(1 + w))
    if r2.operator() != (sym1 + sym2).operator(): # Compact, so return the new result
        return r2
    if len(r2.operands()) < len(r.operands()):  # Smaller, so let's keep this change
        non_w_terms = r2.substitute({w: 0})
        w_terms = r2 - non_w_terms
        w_terms = expand(w_terms.substitute({w: 1}))
        if w_terms.operator() != (sym1 + sym2).operator():
            r3 = expand(non_w_terms - w_terms*(1 + w^2))
            if r3.operator() != (sym1 + sym2).operator():
                return r3
            if len(r3.operands()) < len(r2.operands()):
                return r3
        result = r2
        for t in w_terms.operands():
            r3 = expand(result - t*(1 + w + w^2))
            if r3.operator() != (sym1 + sym2).operator():
                return r3
            if len(r3.operands()) < len(result.operands()):
                result = r3
        if len(result.operands()) < len(r2.operands()):
            return result
        return r2
    if w2_terms == 0:
        non_w_terms = r.substitute({w: 0})
        w_terms = r - non_w_terms
        w_terms = expand(w_terms.substitute({w: 1}))
        r2 = expand(non_w_terms - w_terms*(1 + w^2))
        if r2.operator() != (sym1 + sym2).operator():
            return r2
        if len(r2.operands()) < len(r.operands()):
            return r2
        return r
    return r
    
def layer2_expand_w(r):
    r = w_mod(r)
    r = expand_w(r)
    if r.operator() != (sym1 + sym2).operator():  # Already compact, so return it unchanged
        return r
    test_r = w_mod(w^2*(expand_w(w_mod(w*r))))
    if test_r.operator() != (sym1 + sym2).operator():
        return test_r
    if len(test_r.operands()) < len(r.operands()):
        return test_r
    return r


def matrix_expand_w(A):
    return A.apply_map(expand_w)

# Implement some equations that simplify our results:
var('J3')
var('E p1 p2 p3 q1 q2 q3')
tidy_dict = {}

# tidy_dict['p...q'] = {p1: (ZZ - p2*q1 - p3*q2)/q3}
# tidy_dict['p1*p...q'] = {p1^2: (ZZ - p2*q1 - p3*q2)*p1/q3}
tidy_dict['p...q'] = {p1: (0 - p2*q1 - p3*q2)/q3}
tidy_dict['p1*p...q'] = {p1^2: (0 - p2*q1 - p3*q2)*p1/q3}
tidy_dict['p..q'] = {p1: (0 - p2*q3 - p3*q1)/q2}

# tidy_dict['p...q'] = {d1: (0 - d2*d4 - d3*d5)/d6}
# tidy_dict['p..q'] = {d1: (0 - d2*d6 - d3*d4)/d5}

var('d1 d2 d3')
var('e1 e2 e3')
# tidy_dict['e1'] = {d1^2 : e1 + d2*d3}
# tidy_dict['e2'] = {d2^2 : e2 + d1*d3}
# tidy_dict['e3'] = {d3^2 : e3 + d1*d2}
tidy_dict['e1'] = {d1 : (e1 + d2*d3)/d1}
tidy_dict['e2'] = {d2 : (e2 + d1*d3)/d2}
tidy_dict['e3'] = {d3 : (e3 + d1*d2)/d3}


def tidy(r):
    if r.operator() != (sym1 + sym2).operator():
        return r
    result = r
    for label, the_tidy_dict in tidy_dict.items():
        tmp_result = result.substitute(the_tidy_dict)
        tmp_result = expand(tmp_result)
        if tmp_result.operator() != (sym1 + sym2).operator():
            return tmp_result
        if len(tmp_result.operands()) < len(result.operands()):
            return tmp_result
    return result

def matrix_tidy(A):
    return A.apply_map(tidy)

var('J3 JR3 Z1 Z2 e0 e1 e2 e3 e4 e5 e6')
var('d0 d1 d2 d3 d4 d5 d6')
improved_tidy_dict = {}

# improved_tidy_dict['p..q'] = {p1: (0 - p2*q3 - p3*q1)/q2} # Fix!
# p as Lorentz transform:
# improved_tidy_dict['e1'] = {d1^2 : e1 + d2*d3 + d0*d4}
# improved_tidy_dict['e2'] = {d2^2 : e2 + d1*d3 + d0*d5}
# improved_tidy_dict['e3'] = {d3^2 : e3 + d1*d2 + d0*d6}
# improved_tidy_dict['e4'] = {d4^2 : e4 + d5*d6 + d0*d1}
# improved_tidy_dict['e5'] = {d5^2 : e5 + d4*d6 + d0*d2}
# improved_tidy_dict['e6'] = {d6^2 : e6 + d4*d5 + d0*d3}

# improved_tidy_dict['p...q'] = {p1: (Z1 + p2*q1 + p3*q2)/q3}
# improved_tidy_dict['p1*p...q'] = {p1^2: (Z1 + p2*q1 + p3*q2)*p1/q3}
improved_tidy_dict['e0'] = {d0^2 : e0 + d1*d4 + d2*d5 + d3*d6}

# p as Lorentz transform, but don't chomp the d0 terms:
if not use_rotation:
    improved_tidy_dict['J3'] = {E^3 : J3 - p1^3 - p2^3 - p3^3 + 3*p1*p2*p3 - q1^3 - q2^3 - q3^3 + 3*q1*q2*q3 + 3*E*p1*q1 + 3*E*p2*q2 + 3*E*p3*q3 }
    improved_tidy_dict['p...q'] = {p1: (Z1 - p2*q1 - p3*q2)/q3}
    improved_tidy_dict['p1*p...q'] = {p1^2: (Z1 - p2*q1 - p3*q2)*p1/q3}
    improved_tidy_dict['e1'] = {d1^2 : e1 + d2*d3}
    improved_tidy_dict['e2'] = {d2^2 : e2 + d1*d3}
    improved_tidy_dict['e3'] = {d3^2 : e3 + d1*d2}
    improved_tidy_dict['e4'] = {d4^2 : e4 + d5*d6}
    improved_tidy_dict['e5'] = {d5^2 : e5 + d4*d6}
    improved_tidy_dict['e6'] = {d6^2 : e6 + d4*d5}
    improved_tidy_dict['d...'] = {d1: (Z1 - d2*d4 - d3*d5)/d6}
    improved_tidy_dict['d..'] = {d1: (Z2 - d2*d6 - d3*d4)/d5}

# p as rotation:
# improved_tidy_dict['e0'] = {d0^2 : e0 + d1*d4 + d2*d5 + d3*d6}
# improved_tidy_dict['e1'] = {d1^2 : e1 - d2*d3 + d0*d4}
# improved_tidy_dict['e2'] = {d2^2 : - e2 + d1*d3 - d0*d5}
# improved_tidy_dict['e3'] = {d3^2 : e3 - d1*d2 + d0*d6}
# improved_tidy_dict['e4'] = {d4^2 : e4 - d5*d6 + d0*d1}
# improved_tidy_dict['e5'] = {d5^2 : - e5 + d4*d6 - d0*d2}
# improved_tidy_dict['e6'] = {d6^2 : e6 - d4*d5 + d0*d3}
# improved_tidy_dict['d...'] = {d1: (Z1 + d2*d4 + d3*d5)/d6}
# improved_tidy_dict['d..'] = {d1: (- Z2 - d2*d6 + d3*d4)/d5}

# p as rotation, but don't chomp the d0 terms:
if use_rotation:
    improved_tidy_dict['JR3'] = {E^3 : JR3 - p1^3 + p2^3 - p3^3 - 3*p1*p2*p3 - q1^3 + q2^3 - q3^3 - 3*q1*q2*q3 + 3*E*p1*q1 + 3*E*p2*q2 + 3*E*p3*q3 }
    improved_tidy_dict['p...q'] = {p1: (Z1 + p2*q1 + p3*q2)/q3}
    improved_tidy_dict['p1*p...q'] = {p1^2: (Z1 + p2*q1 + p3*q2)*p1/q3}
    improved_tidy_dict['e1'] = {d1^2 : e1 - d2*d3}
    improved_tidy_dict['e2'] = {d2^2 : - e2 + d1*d3}
    improved_tidy_dict['e3'] = {d3^2 : e3 - d1*d2}
    improved_tidy_dict['e4'] = {d4^2 : e4 - d5*d6}
    improved_tidy_dict['e5'] = {d5^2 : - e5 + d4*d6}
    improved_tidy_dict['e6'] = {d6^2 : e6 - d4*d5}
    improved_tidy_dict['d...'] = {d1: (Z1 + d2*d4 + d3*d5)/d6}
    improved_tidy_dict['d..'] = {d1: (- Z2 - d2*d6 + d3*d4)/d5}


def improved_tidy(r):
    if r.operator() != (sym1 + sym2).operator():  # Already compact, so leave unchanged
        return r
    result = r
    for term in r.operands():
        for elt, tidy_dict in improved_tidy_dict.items():
            working_term = expand(term.substitute(tidy_dict))
            working_result = expand(result - term + working_term)
            if working_result.operator() != (sym1 + sym2).operator():
                result = working_result
                continue
            if len(working_result.operands()) < len(result.operands()):
                result = working_result
                continue
    return result

def matrix_improved_tidy(A):
    return A.apply_map(improved_tidy)


In [16]:
# construct our q = 3 1-form Pauli matrices:
# Lorentz version:
if not use_rotation:
    ZZ3 = matrix([[0,0,0],[0,0,0],[0,0,0]])
    I3 = matrix([[1,0,0],[0,1,0],[0,0,1]])
    T1 = matrix([[0,1,0],[0,0,1],[1,0,0]])
    T2 = matrix([[0,0,w^2],[w,0,0],[0,1,0]])
    T3 = matrix([[1,0,0],[0,w,0],[0,0,w^2]])
    T4 = matrix([[0, 0, w], [w^2,0,0],[0,1,0]])
    Tau1 = matrix_w_mod(T2*T3)
    Tau2 = matrix_w_mod(T3*T1)
    Tau3 = matrix_w_mod(T1*T2)
    Tau4 = matrix([[0,w^2,0],[0,0,w],[1,0,0]])

# Rotation version:
if use_rotation:
    ZZ3 = matrix([[0,0,0],[0,0,0],[0,0,0]])
    I3 = matrix([[1,0,0],[0,1,0],[0,0,1]])
    T1 = matrix([[0,1,0],[0,0,1],[1,0,0]])
    T2 = - matrix([[0,0,w^2],[w,0,0],[0,1,0]])
    T3 = matrix([[1,0,0],[0,w,0],[0,0,w^2]])
    Tau1 = matrix_w_mod(- T2*T3)
    Tau2 = matrix_w_mod(- T3*T1)
    Tau3 = matrix_w_mod(- T1*T2)

test_1 = p1*T1 + p2*T2 + p3*T3
test_2 = q1*Tau1 + q2*Tau2 + q3*Tau3

show('Testing our q = 3 Pauli matrices cube property:')
show('test1:', matrix_expand_w(matrix_w_mod(test_1^3)))
show('test2:', matrix_expand_w(matrix_w_mod(test_2^3)))
show('-------------')

# construct our q = 3 Dirac matrices:
if not use_rotation:
    I9 = I3.tensor_product(I3)
    ZZ9 = ZZ3.tensor_product(ZZ3)
    S0 = T3.tensor_product(I3)

    S1 = w*T2.tensor_product(T1)
    S2 = w*T2.tensor_product(T2)
    S3 = w*T2.tensor_product(T3)

    S4 = w*T1.tensor_product(Tau1)
    S5 = w*T1.tensor_product(Tau2)
    S6 = w*T1.tensor_product(Tau3)

    
# Rotation version:
if use_rotation:
    I9 = I3.tensor_product(I3)
    ZZ9 = ZZ3.tensor_product(ZZ3)
    S0 = T3.tensor_product(I3)

    S1 = - w*T2.tensor_product(T1)
    S2 = - w*T2.tensor_product(T2)
    S3 = - w*T2.tensor_product(T3)

    S4 = w*T1.tensor_product(Tau1)
    S5 = w*T1.tensor_product(Tau2)
    S6 = w*T1.tensor_product(Tau3)

M = S0*S1*S2*S3*S4*S5*S6

# Now w_mod them:
S1 = matrix_w_mod(S1)
S2 = matrix_w_mod(S2)
S3 = matrix_w_mod(S3)
S4 = matrix_w_mod(S4)
S5 = matrix_w_mod(S5)
S6 = matrix_w_mod(S6)

M = matrix_w_mod(M)

show('Our q = 3 Dirac matrices:')
show('S0:', S0)
show('S1:', S1)
show('S2:', S2)
show('S3:', S3)
show('S4:', S4)
show('S5:', S5)
show('S6:', S6)

show('-------------------')
show('Testing B^3:')
B = E*S0 + p1*S1 + p2*S2 + p3*S3 + q1*S4 + q2*S5 + q3*S6
show('B^3|00:', improved_tidy(matrix_expand_w(matrix_w_mod(B^3))[0,0]))
show('B^3:', matrix_improved_tidy(matrix_expand_w(matrix_w_mod(B^3))))

'Testing our q = 3 Pauli matrices cube property:'

'test1:' [p1^3 + p2^3 - 3*p1*p2*p3 + p3^3                               0                               0]
[                              0 p1^3 + p2^3 - 3*p1*p2*p3 + p3^3                               0]
[                              0                               0 p1^3 + p2^3 - 3*p1*p2*p3 + p3^3]

'test2:' [q1^3 + q2^3 - 3*q1*q2*q3 + q3^3                               0                               0]
[                              0 q1^3 + q2^3 - 3*q1*q2*q3 + q3^3                               0]
[                              0                               0 q1^3 + q2^3 - 3*q1*q2*q3 + q3^3]

'-------------'

'Our q = 3 Dirac matrices:'

...

'S6:' [  0   0   0 w^2   0   0   0   0   0]
[  0   0   0   0   w   0   0   0   0]
[  0   0   0   0   0   1   0   0   0]
[  0   0   0   0   0   0 w^2   0   0]
[  0   0   0   0   0   0   0   w   0]
[  0   0   0   0   0   0   0   0   1]
[w^2   0   0   0   0   0   0   0   0]
[  0   w   0   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0   0]

'-------------------'

'Testing B^3:'

'B^3|00:' J3

'B^3:' [         J3    3*E*Z1*w           0     3*Z1*q1 3*Z1*q3*w^2     3*Z1*q2 3*Z1*p2*w^2 3*Z1*p3*w^2   3*Z1*p1*w]
[          0          J3      3*E*Z1     3*Z1*q2 3*Z1*q1*w^2     3*Z1*q3     3*Z1*p1     3*Z1*p2 3*Z1*p3*w^2]
[ 3*E*Z1*w^2           0          J3   3*Z1*q3*w     3*Z1*q2   3*Z1*q1*w 3*Z1*p3*w^2 3*Z1*p1*w^2   3*Z1*p2*w]
[    3*Z1*p2     3*Z1*p3 3*Z1*p1*w^2          J3    3*E*Z1*w           0 3*Z1*q1*w^2   3*Z1*q3*w 3*Z1*q2*w^2]
[  3*Z1*p1*w   3*Z1*p2*w     3*Z1*p3           0          J3      3*E*Z1 3*Z1*q2*w^2   3*Z1*q1*w 3*Z1*q3*w^2]
[    3*Z1*p3     3*Z1*p1 3*Z1*p2*w^2  3*E*Z1*w^2           0          J3     3*Z1*q3 3*Z1*q2*w^2     3*Z1*q1]
[  3*Z1*q1*w     3*Z1*q3   3*Z1*q2*w   3*Z1*p2*w   3*Z1*p3*w     3*Z1*p1          J3    3*E*Z1*w           0]
[  3*Z1*q2*w     3*Z1*q1   3*Z1*q3*w 3*Z1*p1*w^2 3*Z1*p2*w^2   3*Z1*p3*w           0          J3      3*E*Z1]
[3*Z1*q3*w^2   3*Z1*q2*w 3*Z1*q1*w^2   3*Z1*p3*w   3*Z1*p1*w     3*Z1*p2  3*E*Z1*w^2           0          J3]

In [3]:
# Now, let's generate the basis matrices:
import itertools


# Display the dimensions table given d and q, and return the basis_dict:
def dim_table(d, q):
    basis_indices = []
    if d == 3 and q == 2:
        basis_indices = [1,2,3]
    if d == 4 and q == 2:
        basis_indices = [0,1,2,3]
    if d == 3 and q == 3:
        basis_indices = [1,2,3]
    if d == 4 and q == 3:
        basis_indices = [0,1,2,3,4,5,6]
    basis_dict = {}
    k = 0
    while True:
        basis_dict[k] = []
        basis_count = 0
        for elt in itertools.combinations_with_replacement(basis_indices, k):
            match = False
            for b in basis_indices:
                if elt.count(b) >= q:
                    match = True
                    break
            if match:
                continue
            # print(elt)
            basis_dict[k] += [elt]
            basis_count += 1
        k += 1
        if basis_count == 0:
            break
    rows = [['k', 'dim', 'basis']]
    for i in range(k - 1):
        basis_result = basis_dict[i]
        str_basis_result = " ".join("".join(str(x) for x in b) for b in basis_result)
        rows += [[i, len(basis_result), str_basis_result]]
    show(table(rows, header_row=True))
    return basis_dict


# generate the d = 4, q = 3 basis dictionary:
show('The dim = 4, q = 3 k-form dimension table:')
basis4_dict = dim_table(4, 3)


# Extract out all the basis labels:
def generate_basis_labels(basis_dict):
    basis_labels = {}
    for i in basis_dict:
        basis_labels[i] = ['D' + "".join(str(x) for x in b) for b in basis_dict[i]]
    return basis_labels

basis4_labels = generate_basis_labels(basis4_dict)

# Given the matrix indices, return the resulting basis matrix, here for d = 3:
def D3(indices):
    matrices = [T1, T2, T3]
    r = I3
    for i in indices:
        r = r*matrices[i - 1]
    # show(r)
    return matrix_w_mod(expand(r))

# Here for d = 4:
def D4(indices):
    matrices = [S0, S1, S2, S3, S4, S5, S6]
    r = I9
    for i in indices:
        r = r*matrices[i]
    # show(r)
    return matrix_w_mod(expand(r))

# Quick test of basis4_labels:
show('Quick test of basis4 labels, for k = 2:')
show(basis4_labels[2])

# Learn all the d = 4, q = 3, basis matrices:
# NB; makes generate_basis_labels() redundant.
def generate_basis4_matrices(basis_dict):
    basis_labels = {}
    basis_matrices = {}
    for i in basis_dict:
        basis_labels[i] = []
        for b in basis_dict[i]:
            label = 'D' + "".join(str(x) for x in b)
            matrix = D4(list(b))
            # show(list(b), matrix)
            basis_labels[i] += [label]
            basis_matrices[label] = matrix
    return basis_labels, basis_matrices

basis4_labels, basis4_matrices = generate_basis4_matrices(basis4_dict)


# Explore the basis overlap property:
# Test if two matrices are equal up to some w coefficient:
def find_k(A,B):
    for k in range(3):
        if matrix_w_mod(A - w^k*B) == 0:
            return k
        if matrix_w_mod(A + w^k*B) == 0:
            return -k
    return None

# Find the commutator structure for a pair of matrices:
# Ie, A*B = (w)^k B*A
def find_commutator(A,B):
    for k in range(3):
        if matrix_w_mod(A*B - w^k*B*A) == 0:
            return k
    return None

# Get a unique basis given a k:
def get_unique_basis4(k):
    if k not in basis4_labels:
        return
    result = []
    for label in basis4_labels[k]:
        # show(label)
        matrix = basis4_matrices[label]
        match = False
        for A_label in result:
            A = basis4_matrices[A_label]
            # show('A:', A)
            # show(find_k(A, matrix))
            if find_k(A, matrix) is not None:
                match = True
                break
        if not match:
            result += [label]
    return result

# Learn unique basis labels all at once:
def generate_unique_basis4_labels():
    unique_basis_labels = {}
    for k in range(16):
        unique_basis_labels[k] = get_unique_basis4(k)
    return unique_basis_labels

unique_basis4_labels = generate_unique_basis4_labels()

# Quick test:
show('Quick test of unique basis4 labels, for k = 2:')
show(unique_basis4_labels[2])

# Display the unique dimension table:
def unique_dim_table(basis_dict):
    rows = [['k', 'dim', 'unique dim']]
    for k in basis_dict:
        # rows += [[k, len(basis_dict[k]), len(get_unique_basis4(k))]]
        rows += [[k, len(basis_dict[k]), len(unique_basis4_labels[k])]]
    show(table(rows, header_row=True))
    return latex(table(rows, header_row=True))
    
show('----------------------------------')
show('The unique k-form dimension table:')
unique_dim_table(basis4_dict)

'The dim = 4, q = 3 k-form dimension table:'

k,dim,basis
,,
,,0 1 2 3 4 5 6
,,00 01 02 03 04 05 06 11 12 13 14 15 16 22 23 24 25 26 33 34 35 36 44 45 46 55 56 66
,,001 002 003 004 005 006 011 012 013 014 015 016 022 023 024 025 026 033 034 035 036 044 045 046 055 056 066 112 113 114 115 116 122 123 124 125 126 133 134 135 136 144 145 146 155 156 166 223 224 225 226 233 234 235 236 244 245 246 255 256 266 334 335 336 344 345 346 355 356 366 445 446 455 456 466 556 566
,,0011 0012 0013 0014 0015 0016 0022 0023 0024 0025 0026 0033 0034 0035 0036 0044 0045 0046 0055 0056 0066 0112 0113 0114 0115 0116 0122 0123 0124 0125 0126 0133 0134 0135 0136 0144 0145 0146 0155 0156 0166 0223 0224 0225 0226 0233 0234 0235 0236 0244 0245 0246 0255 0256 0266 0334 0335 0336 0344 0345 0346 0355 0356 0366 0445 0446 0455 0456 0466 0556 0566 1122 1123 1124 1125 1126 1133 1134 1135 1136 1144 1145 1146 1155 1156 1166 1223 1224 1225 1226 1233 1234 1235 1236 1244 1245 1246 1255 1256 1266 1334 1335 1336 1344 1345 1346 1355 1356 1366 1445 1446 1455 1456 1466 1556 1566 2233 2234 2235 2236 2244 2245 2246 2255 2256 2266 2334 2335 2336 2344 2345 2346 2355 2356 2366 2445 2446 2455 2456 2466 2556 2566 3344 3345 3346 3355 3356 3366 3445 3446 3455 3456 3466 3556 3566 4455 4456 4466 4556 4566 5566
,,00112 00113 00114 00115 00116 00122 00123 00124 00125 00126 00133 00134 00135 00136 00144 00145 00146 00155 00156 00166 00223 00224 00225 00226 00233 00234 00235 00236 00244 00245 00246 00255 00256 00266 00334 00335 00336 00344 00345 00346 00355 00356 00366 00445 00446 00455 00456 00466 00556 00566 01122 01123 01124 01125 01126 01133 01134 01135 01136 01144 01145 01146 01155 01156 01166 01223 01224 01225 01226 01233 01234 01235 01236 01244 01245 01246 01255 01256 01266 01334 01335 01336 01344 01345 01346 01355 01356 01366 01445 01446 01455 01456 01466 01556 01566 02233 02234 02235 02236 02244 02245 02246 02255 02256 02266 02334 02335 02336 02344 02345 02346 02355 02356 02366 02445 02446 02455 02456 02466 02556 02566 03344 03345 03346 03355 03356 03366 03445 03446 03455 03456 03466 03556 03566 04455 04456 04466 04556 04566 05566 11223 11224 11225 11226 11233 11234 11235 11236 11244 11245 11246 11255 11256 11266 11334 11335 11336 11344 11345 11346 11355 11356 11366 11445 11446 11455 11456 11466 11556 11566 12233 12234 12235 12236 12244 12245 12246 12255 12256 12266 12334 12335 12336 12344 12345 12346 12355 12356 12366 12445 12446 12455 12456 12466 12556 12566 13344 13345 13346 13355 13356 13366 13445 13446 13455 13456 13466 13556 13566 14455 14456 14466 14556 14566 15566 22334 22335 22336 22344 22345 22346 22355 22356 22366 22445 22446 22455 22456 22466 22556 22566 23344 23345 23346 23355 23356 23366 23445 23446 23455 23456 23466 23556 23566 24455 24456 24466 24556 24566 25566 33445 33446 33455 33456 33466 33556 33566 34455 34456 34466 34556 34566 35566 44556 44566 45566
,,001122 001123 001124 001125 001126 001133 001134 001135 001136 001144 001145 001146 001155 001156 001166 001223 001224 001225 001226 001233 001234 001235 001236 001244 001245 001246 001255 001256 001266 001334 001335 001336 001344 001345 001346 001355 001356 001366 001445 001446 001455 001456 001466 001556 001566 002233 002234 002235 002236 002244 002245 002246 002255 002256 002266 002334 002335 002336 002344 002345 002346 002355 002356 002366 002445 002446 002455 002456 002466 002556 002566 003344 003345 003346 003355 003356 003366 003445 003446 003455 003456 003466 003556 003566 004455 004456 004466 004556 004566 005566 011223 011224 011225 011226 011233 011234 011235 011236 011244 011245 011246 011255 011256 011266 011334 011335 011336 011344 011345 011346 011355 011356 011366 011445 011446 011455 011456 011466 011556 011566 012233 012234 012235 012236 012244 012245 012246 012255 012256 012266 012334 012335 012336 012344 012345 012346 012355 012356 012366 012445 012446 012455 012456 012466 012556 012566 013344 013345 013346 013355 013356 013366 013445 013446 013455 013456 013466 013556 013566 014455 014456 014466 014556 014566 015566 022334 0223

'Quick test of basis4 labels, for k = 2:'

['D00',
 'D01',
 'D02',
 'D03',
 'D04',
 'D05',
 'D06',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'D16',
 'D22',
 'D23',
 'D24',
 'D25',
 'D26',
 'D33',
 'D34',
 'D35',
 'D36',
 'D44',
 'D45',
 'D46',
 'D55',
 'D56',
 'D66']

'Quick test of unique basis4 labels, for k = 2:'

['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D15', 'D16']

'----------------------------------'

'The unique k-form dimension table:'

k,dim,unique dim
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


\begin{tabular}{lll}
k & dim & unique dim \\ \hline
$0$ & $1$ & $1$ \\
$1$ & $7$ & $7$ \\
$2$ & $28$ & $9$ \\
$3$ & $77$ & $9$ \\
$4$ & $161$ & $9$ \\
$5$ & $266$ & $9$ \\
$6$ & $357$ & $9$ \\
$7$ & $393$ & $9$ \\
$8$ & $357$ & $9$ \\
$9$ & $266$ & $9$ \\
$10$ & $161$ & $9$ \\
$11$ & $77$ & $9$ \\
$12$ & $28$ & $9$ \\
$13$ & $7$ & $7$ \\
$14$ & $1$ & $1$ \\
$15$ & $0$ & $0$ \\
\end{tabular}

In [4]:
def explore_unique_basis_matrices(k):
    labels = unique_basis4_labels[k]
    for label in labels:
        matrix = basis4_matrices[label]
        show(label, matrix)

show('Unique 2-form basis matrices:')
explore_unique_basis_matrices(2)

def commutation_table(labels):
    show(table([[find_commutator(basis4_matrices[y],basis4_matrices[x]) for y in labels] for x in labels], header_column=[' '] + labels, header_row=labels))
    return latex(table([[find_commutator(basis4_matrices[y],basis4_matrices[x]) for y in labels] for x in labels], header_column=[' '] + labels, header_row=labels))

show('----------------')
show('commutation table for 1-form matrices:')
commutation_table(unique_basis4_labels[1])

'Unique 2-form basis matrices:'

'D00' [  1   0   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0   0]
[  0   0   0 w^2   0   0   0   0   0]
[  0   0   0   0 w^2   0   0   0   0]
[  0   0   0   0   0 w^2   0   0   0]
[  0   0   0   0   0   0   w   0   0]
[  0   0   0   0   0   0   0   w   0]
[  0   0   0   0   0   0   0   0   w]

'D01' [0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0]

'D02' [  0   0   0   0   0   0   0   0 w^2]
[  0   0   0   0   0   0   w   0   0]
[  0   0   0   0   0   0   0   1   0]
[  0   0 w^2   0   0   0   0   0   0]
[  w   0   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0   0]
[  0   0   0   0   0 w^2   0   0   0]
[  0   0   0   w   0   0   0   0   0]
[  0   0   0   0   1   0   0   0   0]

'D03' [  0   0   0   0   0   0   1   0   0]
[  0   0   0   0   0   0   0   w   0]
[  0   0   0   0   0   0   0   0 w^2]
[  1   0   0   0   0   0   0   0   0]
[  0   w   0   0   0   0   0   0   0]
[  0   0 w^2   0   0   0   0   0   0]
[  0   0   0   1   0   0   0   0   0]
[  0   0   0   0   w   0   0   0   0]
[  0   0   0   0   0 w^2   0   0   0]

...

'D16' [  0   w   0   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0   0]
[w^2   0   0   0   0   0   0   0   0]
[  0   0   0   0   1   0   0   0   0]
[  0   0   0   0   0 w^2   0   0   0]
[  0   0   0   w   0   0   0   0   0]
[  0   0   0   0   0   0   0 w^2   0]
[  0   0   0   0   0   0   0   0   w]
[  0   0   0   0   0   0   1   0   0]

'----------------'

'commutation table for 1-form matrices:'

,D0,D1,D2,D3,D4,D5,D6
D0,,,,,,,
D1,,,,,,,
D2,,,,,,,
D3,,,,,,,
D4,,,,,,,
D5,,,,,,,
D6,,,,,,,


\begin{tabular}{l|lllllll}
  & D0 & D1 & D2 & D3 & D4 & D5 & D6 \\ \hline
D0 & $0$ & $2$ & $2$ & $2$ & $1$ & $1$ & $1$ \\
D1 & $1$ & $0$ & $1$ & $2$ & $2$ & $1$ & $0$ \\
D2 & $1$ & $2$ & $0$ & $1$ & $0$ & $2$ & $1$ \\
D3 & $1$ & $1$ & $2$ & $0$ & $1$ & $0$ & $2$ \\
D4 & $2$ & $1$ & $0$ & $2$ & $0$ & $1$ & $2$ \\
D5 & $2$ & $2$ & $1$ & $0$ & $2$ & $0$ & $1$ \\
D6 & $2$ & $0$ & $2$ & $1$ & $1$ & $2$ & $0$ \\
\end{tabular}

In [5]:
# Now, start to classify our k = 2 matrices

# Find quadrant for a matrix:
# Define our 9x9 permutation matrix:
P9 = block_matrix([[ZZ3,I3,ZZ3],[ZZ3,ZZ3,I3],[I3,ZZ3,ZZ3]])

# Define our 9x9 quadrant matrices:
Q1 = block_matrix([[I3,ZZ3,ZZ3],[ZZ3,ZZ3,ZZ3],[ZZ3,ZZ3,ZZ3]])
Q2 = block_matrix([[ZZ3,ZZ3,ZZ3],[ZZ3,I3,ZZ3],[ZZ3,ZZ3,ZZ3]])
Q3 = block_matrix([[ZZ3,ZZ3,ZZ3],[ZZ3,ZZ3,ZZ3],[ZZ3,ZZ3,I3]])

# Given a matrix, find the corresponding non-zero quadrants:
def get_quadrants(A):
    quadrants = []
    if Q1*A*Q1 != 0:
        quadrants += [1]
    if Q2*A*Q2 != 0:
        quadrants += [5]
    if Q3*A*Q3 != 0:
        quadrants += [9]
    if Q2*P9^2*A*Q2 != 0:
        quadrants += [2]
    if Q3*P9^2*A*Q3 != 0:
        quadrants += [6]
    if Q1*P9^2*A*Q1 != 0:
        quadrants += [7]
    if Q3*P9*A*Q3 != 0:
        quadrants += [3]    
    if Q1*P9*A*Q1 != 0:
        quadrants += [4]    
    if Q2*P9*A*Q2 != 0:
        quadrants += [8]
    return quadrants

# Quick test:
# get_quadrants(basis4_matrices['D002'])

# Find quadrants:
def get_all_quadrants(k):
    basis_labels = unique_basis4_labels[k]
    for label in basis_labels:
        show(label, get_quadrants(basis4_matrices[label]))

# Now display the quadrants:
show('The quadrants for the unique k = 2 basis matrices:')
get_all_quadrants(2)

# Now classify them based on [1,5,9], [3,4,8], [2,6,7]:
def classify_basis_matrices(k):
    class1 = []
    class2 = []
    class3 = []
    basis_labels = unique_basis4_labels[k]
    for label in basis_labels:
        quadrants = get_quadrants(basis4_matrices[label])
        if quadrants == [1,5,9]:
            class1 += [label]
        if quadrants == [3,4,8]:
            class2 += [label]
        if quadrants == [2,6,7]:
            class3 += [label]
    show('Class 1:', class1)
    show('Class 2:', class2)
    show('Class 3:', class3)
    return class1, class2, class3

# show('Classes for k = 3 unique basis matrices:')
# classify_basis_matrices(3)

show('Classes for k = 2 unique basis matrices:')
classify_basis_matrices(2)

'The quadrants for the unique k = 2 basis matrices:'

'D00' [1, 5, 9]

'D01' [3, 4, 8]

'D02' [3, 4, 8]

'D03' [3, 4, 8]

...

'Classes for k = 2 unique basis matrices:'

'Class 1:' ['D00', 'D15', 'D16']

'Class 2:' ['D01', 'D02', 'D03']

'Class 3:' ['D04', 'D05', 'D06']

(['D00', 'D15', 'D16'], ['D01', 'D02', 'D03'], ['D04', 'D05', 'D06'])

In [10]:
# Let's write the custum_latex(r) code:
# Converts an expression into correctly sorted and formatted latex

# First, define all the variables:
var('d0 d1 d2 d3 d4 d5 d6')
var('e0 e1 e2 e3 e4 e5 e6')
var('A0 A1 A2 A3 A4 A5 A6')
var('E1 E2 E3')
var('B1 B2 B3')
var('C1 C2 C3')

# The order of the terms in this list, is the order they will be sorted to in expressions:
custom_latex_sort_order = [-1, w, w^2]
custom_latex_sort_order += [d0, d1, d2, d3, d4, d5, d6]
custom_latex_sort_order += [e0, e1, e2, e3, e4, e5, e6]
custom_latex_sort_order += [A0, A1, A2, A3, A4, A5, A6]
custom_latex_sort_order += [E1, E2, E3, B1, B2, B3, C1, C2, C3]

# Maps indecies to their associated latex:
custom_latex_string = ['-', 'w', 'w^2']
custom_latex_string += [r'\partial_0', r'\partial_1', r'\partial_2', r'\partial_3', r'\partial_4', r'\partial_5', r'\partial_6']
custom_latex_string += ['e_0', 'e_1', 'e_2', 'e_3', 'e_4', 'e_5', 'e_6']
custom_latex_string += ['A_0', 'A_1', 'A_2', 'A_3', 'A_4', 'A_5', 'A_6']
custom_latex_string += ['E_1', 'E_2', 'E_3', 'B_1', 'B_2', 'B_3', 'C_1', 'C_2', 'C_3']

# The function itself:
def custom_latex(r):
    r = expand(r)
    latex_result = ''
    if r.operator() == (sym1 * sym2).operator():
        # show('r.operands:', r.operands())
        match = False
        for x in r.operands():
            if x < 0:
                match = True
        if match:
            # show('Match!')
            latex_result += ' - ' + custom_latex(-r)
        else:
            idx_list = []
            power_term_tails = {}
            number = 1
            for t in r.operands():
                # show('t:', t)
                if t.operator() == (sym1^sym2).operator():
                    head, tail = t.operands()
                    idx = custom_latex_sort_order.index(head)
                    idx_list.append(idx)
                    power_term_tails[idx] = tail
                else:
                    if t not in custom_latex_sort_order:
                        number = t
                    else:
                        idx = custom_latex_sort_order.index(t)
                        idx_list.append(idx)
            idx_list.sort()
            result_list = []
            if number != 1:
                result_list.append(latex(number))
            for i in idx_list:
                if i in power_term_tails:
                    result_list.append(custom_latex_string[i] + '^{' + latex(power_term_tails[i]) + '}')
                else:
                    result_list.append(custom_latex_string[i])
            latex_result += ' '.join(x for x in result_list)
    elif r.operator() == (sym1 + sym2).operator():
        latex_result += ' + '.join(custom_latex(term) for term in r.operands())
    elif r.operator() == (sym1^sym2).operator():
        head, tail = r.operands()
        head_idx = custom_latex_sort_order.index(head)
        latex_result += '%s^{%s}' % (custom_latex_string[head_idx], latex(tail))
    else:
        idx = custom_latex_sort_order.index(r)
        latex_result += custom_latex_string[idx]
    return latex_result
    # return latex_result.replace(' + - ', ' - ')
            
# Quick test:    
# custom_latex(2*w)
# custom_latex(- 2* w*d5^7)
show('Quick test of our custom_latex() function:')
print(custom_latex(A5*d3*w^2 + A2*d4 - 3.14* d5^7 + w + A3 + w^2))

'Quick test of our custom_latex() function:'

 - 3.14000000000000 \partial_5^{ 7 } + w^{ 2 } \partial_3 A_5 + \partial_4 A_2 + w^{2} + A_3 + w


In [11]:
# Now extract out basis terms:
def get_basis4(k, A, use_improved_tidy = True, use_tidy = False, use_Z_zero = False):
    if k not in unique_basis4_labels:
        return
    labels = unique_basis4_labels[k]
    latex_result = ''
    results = {}
    for label in labels:
        matrix = basis4_matrices[label]
        term = (expand(A*matrix^5)).trace()/9
        term = w_mod(term)
        # term = expand_w(term)
        term = layer2_expand_w(term)
        if use_tidy:
            term = tidy(term)
        if use_improved_tidy:
            term = improved_tidy(term)
        term = expand(term)
        # term = expand_w(term)
        term = layer2_expand_w(term)
        if use_Z_zero:
            term = term.substitute({Z2: 0})
        if term != 0:
            show(label, '::', term)
            results[label] = term
            latex_result += '& ' + latex(label + ': ') + custom_latex(term) + '\\\\\n'
    print(latex_result)
    return results

# Define some variables:
var('d0 d1 d2 d3 d4 d5 d6')
var('A0 A1 A2 A3 A4 A5 A6')
d = d0*S0 + d1*S1 + d2*S2 + d3*S3 + d4*S4 + d5*S5 + d6*S6
A = A0*S0 + A1*S1 + A2*S2 + A3*S3 + A4*S4 + A5*S5 + A6*S6


show('---------------------')
show('Quick test of get_basis4(1, A):')
get_basis4(1, A)

show('----------------------')
show('The terms of G = dA:')
get_basis4(2, d*A)

show('----------------------')
show('The terms of dd:')
get_basis4(2, d*d)

'---------------------'

'Quick test of get_basis4(1, A):'

'D0' '::' A0

'D1' '::' A1

'D2' '::' A2

...

'D06' '::' -d0*d6*w^2 + e3*w^2

'D15' '::' -Z2*w^2

'D16' '::' 2*Z1

& \text{\texttt{D00:{ }}} e_0 \\
 & \text{\texttt{D01:{ }}} - w \partial_0 \partial_1 + w e_4 \\
 & \text{\texttt{D02:{ }}} - w \partial_0 \partial_2 + w e_5 \\
 & \text{\texttt{D03:{ }}} - w \partial_0 \partial_3 + w e_6 \\
 & \text{\texttt{D04:{ }}} - w^{ 2 } \partial_0 \partial_4 + w^{ 2 } e_1 \\
 & \text{\texttt{D05:{ }}} - w^{ 2 } \partial_0 \partial_5 + w^{ 2 } e_2 \\
 & \text{\texttt{D06:{ }}} - w^{ 2 } \partial_0 \partial_6 + w^{ 2 } e_3 \\
 & \text{\texttt{D15:{ }}} - Z_{2} w^{ 2 } \\
 & \text{\texttt{D16:{ }}} 2 \\



{'D00': e0,
 'D01': -d0*d1*w + e4*w,
 'D02': -d0*d2*w + e5*w,
 'D03': -d0*d3*w + e6*w,
 'D04': -d0*d4*w^2 + e1*w^2,
 'D05': -d0*d5*w^2 + e2*w^2,
 'D06': -d0*d6*w^2 + e3*w^2,
 'D15': -Z2*w^2,
 'D16': 2*Z1}

In [13]:
# Now define the w-inverse:
def w_inverse(r):
    r = w_mod(expand(r))
    if r == 0:
        return r
    if r.operator() != (sym1 + sym2).operator():
        term = r.substitute({w: 1})
        coeff = expand(r/term)
        if coeff == 1:
            return r
        if coeff == w:
            return w_mod(expand(w^2*term))
        if coeff == w^2:
            return w_mod(expand(w*term))
        return
    if r.operator() == (sym1 + sym2).operator():
        result = 0
        for term in r.operands():
            result += w_inverse(term)
        return result

# Now define k-form Hodge star:
def Hstar_k(k,q,A, use_improved_tidy = False):
    if k not in unique_basis4_labels:
        return
    labels = unique_basis4_labels[k]
    if False and k == 2 and q == 3:
        # labels = ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D14', 'D15', 'D16']
        labels = ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D41', 'D15', 'D16']
    r = ZZ9
    for label in labels:
        matrix = basis4_matrices[label]
        term = (expand(A*matrix^2)).trace()/9
        term = expand(term)
        term = w_mod(term)
        # term = expand_w(term)
        if use_improved_tidy:
            term = improved_tidy(term)
        if term != 0:
            if q == 2:
                term = w_inverse(term)
                r += matrix^(3 - q + 1)*w*M^2*term
            if q == 3:
                r += matrix^(6 - q + 1)*w*M^2*term
    # return matrix_expand_w(matrix_w_mod(expand(r)))
    return matrix_w_mod(expand(r))

show('Quick test of the Hodge star operator:')
show('A A ** A:')
get_basis4(14, A*A*Hstar_k(1, 3, A))
show('A * A:')
get_basis4(14, A*Hstar_k(1, 2, A))

'Quick test of the Hodge star operator:'

'A A ** A:'

'D00112233445566' '::' A0^3 + A1^3 + A2^3 - 3*A1*A2*A3 + A3^3 - 3*A0*A1*A4 + A4^3 - 3*A0*A2*A5 + A5^3 - 3*A0*A3*A6 - 3*A4*A5*A6 + A6^3

& \text{\texttt{D00112233445566:{ }}} A_0^{3} + A_1^{3} + A_2^{3} +  - 3 A_1 A_2 A_3 + A_3^{3} +  - 3 A_0 A_1 A_4 + A_4^{3} +  - 3 A_0 A_2 A_5 + A_5^{3} +  - 3 A_0 A_3 A_6 +  - 3 A_4 A_5 A_6 + A_6^{3} \\



'A * A:'

'D00112233445566' '::' A0^2 + A1^2 + A2^2 + A3^2 + A4^2 + A5^2 + A6^2

& \text{\texttt{D00112233445566:{ }}} A_0^{2} + A_1^{2} + A_2^{2} + A_3^{2} + A_4^{2} + A_5^{2} + A_6^{2} \\



{'D00112233445566': A0^2 + A1^2 + A2^2 + A3^2 + A4^2 + A5^2 + A6^2}

In [15]:
# Given an expression, extract out the e_i terms:
# Note, currently breaks if r is not in terms of e_i and d_i.
def extract_e_terms(r, prefix=''):
    r = expand(r)
    r = w_mod(r)
    data = {}
    data['rest'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 0, e5: 0, e6: 0})
    data['e0'] = r.substitute({e0: 1, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 0, e5: 0, e6: 0, d0: 0})
    data['Z1'] = r.substitute({e0: 0, Z1: 1, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 0, e5: 0, e6: 0, d0: 0})
    data['Z2'] = r.substitute({e0: 0, Z1: 0, Z2: 1, e1: 0, e2: 0, e3: 0, e4: 0, e5: 0, e6: 0, d0: 0})
    data['e1'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 1, e2: 0, e3: 0, e4: 0, e5: 0, e6: 0, d0: 0})
    data['e2'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 1, e3: 0, e4: 0, e5: 0, e6: 0, d0: 0})
    data['e3'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 1, e4: 0, e5: 0, e6: 0, d0: 0})
    data['e4'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 1, e5: 0, e6: 0, d0: 0})
    data['e5'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 0, e5: 1, e6: 0, d0: 0})
    data['e6'] = r.substitute({e0: 0, Z1: 0, Z2: 0, e1: 0, e2: 0, e3: 0, e4: 0, e5: 0, e6: 1, d0: 0})
    # data['d0'] = data['rest'].substitute({d0: 1})
    data['d0d1'] = data['rest'].substitute({d0: 1, d1: 1, d2: 0, d3: 0, d4: 0, d5: 0, d6: 0})
    data['d0d2'] = data['rest'].substitute({d0: 1, d1: 0, d2: 1, d3: 0, d4: 0, d5: 0, d6: 0})
    data['d0d3'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 1, d4: 0, d5: 0, d6: 0})
    data['d0d4'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 1, d5: 0, d6: 0})
    data['d0d5'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 0, d5: 1, d6: 0})
    data['d0d6'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 0, d5: 0, d6: 1})
    data['rest'] = data['rest'].substitute({d0: 0})
    
    # data['d0d1'] = data['rest'].substitute({d0*d1: 1, d0*d2: 0, d0*d3: 0, d0*d4: 0, d0*d5: 0, d0*d6: 0})
    # data['d0d2'] = data['rest'].substitute({d0*d1: 0, d0*d2: 1, d0*d3: 0, d0*d4: 0, d0*d5: 0, d0*d6: 0})
    # data['d0d3'] = data['rest'].substitute({d0*d1: 0, d0*d2: 0, d0*d3: 1, d0*d4: 0, d0*d5: 0, d0*d6: 0})
    # data['d0d4'] = data['rest'].substitute({d0*d1: 0, d0*d2: 0, d0*d3: 0, d0*d4: 1, d0*d5: 0, d0*d6: 0})
    # data['d0d5'] = data['rest'].substitute({d0*d1: 0, d0*d2: 0, d0*d3: 0, d0*d4: 0, d0*d5: 1, d0*d6: 0})
    # data['d0d6'] = data['rest'].substitute({d0*d1: 0, d0*d2: 0, d0*d3: 0, d0*d4: 0, d0*d5: 0, d0*d6: 1})
    # data['rest'] = data['rest'].substitute({d0*d1: 0, d0*d2: 0, d0*d3: 0, d0*d4: 0, d0*d5: 0, d0*d6: 0})
    
    # data['d1'] = data['rest'].substitute({d0: 1, d1: 1, d2: 0, d3: 0, d4: 0, d5: 0, d6: 0}) # Slightly broken!
    # data['d2'] = data['rest'].substitute({d0: 1, d1: 0, d2: 1, d3: 0, d4: 0, d5: 0, d6: 0}) # Assumes d0*di terms.
    # data['d3'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 1, d4: 0, d5: 0, d6: 0})
    # data['d4'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 1, d5: 0, d6: 0})
    # data['d5'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 0, d5: 1, d6: 0})
    # data['d6'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 0, d5: 0, d6: 1})
    # data['rest'] = data['rest'].substitute({d0: 1, d1: 0, d2: 0, d3: 0, d4: 0, d5: 0, d6: 0})
    
    e_coeff = data['e1'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    e_coeff += data['e2'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    e_coeff += data['e3'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    e_vec_1 = custom_latex(w_mod(w_inverse(e_coeff)*data['e1']))
    e_vec_2 = custom_latex(w_mod(w_inverse(e_coeff)*data['e2']))
    e_vec_3 = custom_latex(w_mod(w_inverse(e_coeff)*data['e3']))
    
    eb_coeff = data['e4'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    eb_coeff += data['e5'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    eb_coeff += data['e6'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    eb_vec_1 = custom_latex(w_mod(w_inverse(eb_coeff)*data['e4']))
    eb_vec_2 = custom_latex(w_mod(w_inverse(eb_coeff)*data['e5']))
    eb_vec_3 = custom_latex(w_mod(w_inverse(eb_coeff)*data['e6']))
    
    d0_del_coeff = data['d0d1'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_del_coeff += data['d0d2'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_del_coeff += data['d0d3'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_del_vec_1 = custom_latex(w_mod(w_inverse(d0_del_coeff)*data['d0d1']))
    d0_del_vec_2 = custom_latex(w_mod(w_inverse(d0_del_coeff)*data['d0d2']))
    d0_del_vec_3 = custom_latex(w_mod(w_inverse(d0_del_coeff)*data['d0d3']))
    
    d0_delb_coeff = data['d0d4'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_delb_coeff += data['d0d5'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_delb_coeff += data['d0d6'].substitute({E1: 1, B1: 1, C1: 1, E2: 0, E3: 0, B2: 0, B3: 0, C2: 0, C3: 0})
    d0_delb_vec_1 = custom_latex(w_mod(w_inverse(d0_delb_coeff)*data['d0d4']))
    d0_delb_vec_2 = custom_latex(w_mod(w_inverse(d0_delb_coeff)*data['d0d5']))
    d0_delb_vec_3 = custom_latex(w_mod(w_inverse(d0_delb_coeff)*data['d0d6']))
    
    if e_coeff == 1:
        e_coeff = ''
    if eb_coeff == 1:
        eb_coeff = ''
    if d0_del_coeff == 1:
        d0_del_coeff = ''
    if d0_delb_coeff == 1:
        d0_delb_coeff = ''
    
    if e_coeff == -1:
        e_coeff = '-'
    if eb_coeff == -1:
        eb_coeff = '-'
    if d0_del_coeff == -1:
        d0_del_coeff = '-'
    if d0_delb_coeff == -1:
        d0_delb_coeff = '-'
    
    
    expr = ''
    if prefix != '':
        expr += prefix + r': '
    expr += r'\bar{Z}\cdot(%s,%s,%s)' % (data['e0'], custom_latex(data['Z1']), custom_latex(data['Z2']))
    expr += r' + %s e\cdot(%s,%s,%s)' % (e_coeff, e_vec_1, e_vec_2, e_vec_3)
    expr += r' + %s \bar{e}\cdot(%s,%s,%s)' % (eb_coeff, eb_vec_1, eb_vec_2, eb_vec_3)
    if data['d0d1'] != 0:
        # expr += r'\\ + \partial_0\nabla\cdot(%s,%s,%s)' % (data['d0d1'], data['d0d2'], data['d0d3'])
        # expr += r'\\ + %s \partial_0\nabla\cdot(%s,%s,%s)' % (d0_del_coeff, d0_del_vec_1, d0_del_vec_2, d0_del_vec_3)
        expr += r' + %s \partial_0\nabla\cdot(%s,%s,%s)' % (d0_del_coeff, d0_del_vec_1, d0_del_vec_2, d0_del_vec_3)
    # if data['d0d4'] != 0:
        # expr += r' + \partial_0\bar{\nabla}\cdot(%s,%s,%s)' % (data['d0d4'], data['d0d5'], data['d0d6'])
        expr += r' + %s \partial_0\bar{\nabla}\cdot(%s,%s,%s)' % (d0_delb_coeff, d0_delb_vec_1, d0_delb_vec_2, d0_delb_vec_3)
    if data['rest'] != 0:
        expr += r'\\ + %s' % data['rest']
    expr = expr.replace('+ -', '-')
    show(LatexExpr(expr))
    # return data
    return LatexExpr(expr)

# Quick test:
# extract_e_terms(w^2*B1_eqn, 'B1')

var('E1 E2 E3')
var('B1 B2 B3')
var('C1 C2 C3')

# Define our Maxwell matrix:
G = E1*S0*S1 + E2*S0*S2 + E3*S0*S3
G += B1*S0*S4 + B2*S0*S5 + B3*S0*S6
G += C1*S0^2 + w^2*C2*S1*S5 + C3*S1*S6

show('Maxwell equations as 1-forms:')
test_improved_tidy_eqns = get_basis4(1, d*d*G)

show('Maxwell equations as 4-forms:')
eqns_dim_4 = get_basis4(4, d*d*G)

show('--------------------------------------------')
show('The extracted out q = 3 Maxwell equations:')
C1_latex = extract_e_terms(test_improved_tidy_eqns['D0'], 'C1')
C2_latex = extract_e_terms(w*eqns_dim_4['D0015'], 'C2')
C3_latex = extract_e_terms(eqns_dim_4['D0016'], 'C3')
show('   ')
E1_latex = extract_e_terms(test_improved_tidy_eqns['D1'], 'E1')
E2_latex = extract_e_terms(test_improved_tidy_eqns['D2'], 'E2')
E3_latex = extract_e_terms(test_improved_tidy_eqns['D3'], 'E3')
show('   ')
B1_latex = extract_e_terms(test_improved_tidy_eqns['D4'], 'B1')
B2_latex = extract_e_terms(test_improved_tidy_eqns['D5'], 'B2')
B3_latex = extract_e_terms(test_improved_tidy_eqns['D6'], 'B3')

'Maxwell equations as 1-forms:'

'D0' '::' -B1*d0*d1*w^2 - B2*d0*d2*w^2 - B3*d0*d3*w^2 - E1*d0*d4*w - E2*d0*d5*w - E3*d0*d6*w + B1*e4*w^2 + B2*e5*w^2 + B3*e6*w^2 + E1*e1*w + E2*e2*w + E3*e3*w + 2*C2*Z1 - C3*Z2 + C1*e0

'D1' '::' -C1*d0*d1*w^2 - B3*d0*d5*w^2 - C2*d0*d2*w - B1*d0*d4*w + 2*E3*Z1*w^2 + B3*e2*w^2 + C1*e4*w^2 - C3*d0*d3 - B2*d0*d6 - E2*Z2*w + B1*e1*w + C2*e5*w + E1*e0 + B2*e3 + C3*e6

'D2' '::' -C1*d0*d2*w^2 - B1*d0*d6*w^2 - C2*d0*d3*w - B2*d0*d5*w + 2*E1*Z1*w^2 + B1*e3*w^2 + C1*e5*w^2 - C3*d0*d1 - B3*d0*d4 - E3*Z2*w + B2*e2*w + C2*e6*w + E2*e0 + B3*e1 + C3*e4

'D3' '::' -C1*d0*d3*w^2 - B2*d0*d4*w^2 - C2*d0*d1*w - B3*d0*d6*w + 2*E2*Z1*w^2 + B2*e1*w^2 + C1*e6*w^2 - C3*d0*d2 - B1*d0*d5 - E1*Z2*w + B3*e3*w + C2*e4*w + E3*e0 + B1*e2 + C3*e5

...

E3: \bar{Z}\cdot(E3,2 w^{ 2 } E_2, - w E_1) +  e\cdot(w^{ 2 } B_2,B_1,w B_3) + w^2 \bar{e}\cdot(w^{ 2 } C_2,w C_3,C_1) -w^2 \partial_0\nabla\cdot(w^{ 2 } C_2,w C_3,C_1) - \partial_0\bar{\nabla}\cdot(w^{ 2 } B_2,B_1,w B_3)

'   '

B1: \bar{Z}\cdot(B1,2 w B_2, - w^{ 2 } B_3) + w e\cdot(C_1,w^{ 2 } C_3,w C_2) + w^2 \bar{e}\cdot(E_1,w E_3,w^{ 2 } E_2) -w^2 \partial_0\nabla\cdot(E_1,w E_3,w^{ 2 } E_2) -w \partial_0\bar{\nabla}\cdot(C_1,w^{ 2 } C_3,w C_2)

B2: \bar{Z}\cdot(B2,2 w B_3, - w^{ 2 } B_1) + w e\cdot(w C_2,C_1,w^{ 2 } C_3) +  \bar{e}\cdot(w E_3,w^{ 2 } E_2,E_1) - \partial_0\nabla\cdot(w E_3,w^{ 2 } E_2,E_1) -w \partial_0\bar{\nabla}\cdot(w C_2,C_1,w^{ 2 } C_3)

B3: \bar{Z}\cdot(B3,2 w B_1, - w^{ 2 } B_2) + w e\cdot(w^{ 2 } C_3,w C_2,C_1) + w \bar{e}\cdot(w^{ 2 } E_2,E_1,w E_3) -w \partial_0\nabla\cdot(w^{ 2 } E_2,E_1,w E_3) -w \partial_0\bar{\nabla}\cdot(w^{ 2 } C_3,w C_2,C_1)

In [17]:
# Given a 3x3 matrix, extract the Pauli matrix coefficients:
def get_pauli_basis(A):
    labels = ['I', 'T1', 'T2', 'T3', 'Tau1', 'Tau2', 'Tau3', 'T4', 'Tau4']
    matrices = [I3, T1, T2, T3, Tau1, Tau2, Tau3, T4, Tau4]
    indecies = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    results = []
    for idx in range(len(labels)):
        matrix = matrices[idx]
        term = (matrix^2*A).trace()/3
        term = expand(term)
        term = w_mod(term)
        term = expand_w(term)
        if term != 0:
            results += [(idx, term)]
            label = labels[idx]
            index = indecies[idx]
            show(label, ':: ', term)
    return results